In [13]:
# pip install graphviz

In [14]:
# pip install pygraphviz --global-option=build_ext --global-option="-IC:\Users\saraw\Documents\langgraph\.venv\Lib\site-packages\graphviz\include" --global-option="-LC:\Users\saraw\Documents\langgraph\.venv\Lib\site-packages\graphviz\lib"

In [1]:
from dotenv import load_dotenv

In [2]:
_ = load_dotenv()

In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [18]:
tool = TavilySearchResults(max_results=4)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [19]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [20]:
class Agent:
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:
                print("\n ....bad tool name....")
                result = "bad tool name, retry"
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [21]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt)

In [15]:
# from IPython.display import Image

# Image(abot.graph.get_graph().draw_png())

In [22]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_r8nCwzRz3KjtPE5qxNgIvoam', 'type': 'tool_call'}
Back to the model!


In [23]:
result

{'messages': [HumanMessage(content='What is the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_r8nCwzRz3KjtPE5qxNgIvoam', 'function': {'arguments': '{"query":"San Francisco weather today"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 151, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-808144e8-6ce3-46e0-bc31-1bf13cbb12e7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_r8nCwzRz3KjtPE5qxNgIvoam', 'type': 'tool_call'}],

In [26]:
result['messages'][-1].content

'The current weather in San Francisco is overcast with a temperature of 13.3°C (55.9°F). The wind is coming from the south at 8.3 mph (13.3 kph), and the humidity is at 75%. The atmospheric pressure is 1011 mb.'

In [27]:
query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': 'call_TtW2BdSIIIysL8C9IwQuSpr3', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Kansas City Chiefs headquarters location'}, 'id': 'call_do32SyVk9HjKjyTnrABC23Kn', 'type': 'tool_call'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Missouri GDP 2024'}, 'id': 'call_dmCMCrAPkKNoTOWjT1mYC4fh', 'type': 'tool_call'}
Back to the model!


In [31]:
result['messages'][-1].content

"1. **Who won the Super Bowl in 2024?**  \n   The Kansas City Chiefs won the Super Bowl in 2024.\n\n2. **In what state is the winning team headquarters located?**  \n   The Kansas City Chiefs are headquartered in Kansas City, Missouri.\n\n3. **What is the GDP of that state?**  \n   In 2024, Missouri's GDP reached $349.2 billion."